In [44]:
import re
import json
import pandas as pd
import folium
from folium.plugins import HeatMap, Fullscreen
from folium.plugins import AntPath
from pandas import json_normalize
import requests
import time

### LECTURA Y VALIDACION DE LLAVES PRINCIPALES

In [2]:
# Reemplaza 'tu_archivo.json' por el nombre real de tu archivo
with open(r"C:\Users\9 ----- SIG\Downloads\Rutas.json", 'r', encoding='utf-8') as f:
    datos = json.load(f)

# Ahora 'datos' es un diccionario o lista de Python
print("¡Archivo cargado con éxito!")

¡Archivo cargado con éxito!


In [3]:
print(f"Tipo de estructura raíz: {type(datos)}")

if isinstance(datos, dict):
    print("Llaves principales:", datos.keys())
elif isinstance(datos, list):
    print(f"Es una lista con {len(datos)} elementos.")
    print("Primer elemento:", datos[0] if datos else "Lista vacía")

Tipo de estructura raíz: <class 'dict'>
Llaves principales: dict_keys(['semanticSegments', 'rawSignals', 'userLocationProfile'])


### CREACIÓN DE DATAFRAMES, LIMPIEZA DE COLUMNAS Y EXTRACCIÓN DE LATITUD Y LONGITUD

In [4]:
# 2. Extraer 'semanticSegments' del JSON 
segments = datos.get('semanticSegments', [])

# 3. Crear el DataFrame de Visitas (donde te detuviste)
visitas = [s for s in segments if 'visit' in s]
df_visitas = json_normalize(visitas)
df_visitas['startTime'] = pd.to_datetime(df_visitas['startTime'])
df_visitas['endTime'] = pd.to_datetime(df_visitas['endTime'])

# 4. Crear el DataFrame de Actividad (tus rutas/viajes)
actividades = [s for s in segments if 'activity' in s]
df_actividades = json_normalize(actividades)
df_actividades['startTime'] = pd.to_datetime(df_visitas['startTime'])
df_actividades['endTime'] = pd.to_datetime(df_visitas['endTime'])

print(f"Visitas encontradas: {len(df_visitas)}")
print(f"Rutas encontradas: {len(df_actividades)}")

Visitas encontradas: 208
Rutas encontradas: 211


In [5]:
# Columnas que realmente nos cuentan algo
cols_actividades = [
    'startTime', 'endTime', 'activity.start.latLng', 'activity.end.latLng', 
    'activity.distanceMeters', 'activity.topCandidate.type'
]

cols_visitas = [
    'startTime', 'endTime', 'visit.topCandidate.placeLocation.latLng'
]
# Filtrar y renombrar para que sea más humano
df_actividades = df_actividades[cols_actividades].copy()
df_visitas = df_visitas[cols_visitas].copy()

In [6]:
def limpiar_string_coordenadas(texto):
    if pd.isna(texto) or not isinstance(texto, str):
        return None, None
    
    # Busca números decimales (ej: 4.609 o -74.15)
    numeros = re.findall(r"[-+]?\d*\.\d+|\d+", texto)
    
    if len(numeros) >= 2:
        return float(numeros[0]), float(numeros[1])
    return None, None

# Aplicamos la lógica de forma eficiente
# Para Visitas
df_visitas[['latitud', 'longitud']] = df_visitas['visit.topCandidate.placeLocation.latLng'].apply(
    lambda x: pd.Series(limpiar_string_coordenadas(x))
)

# Para Actividades (Origen)
df_actividades[['lat_origen', 'lng_origen']] = df_actividades['activity.start.latLng'].apply(
    lambda x: pd.Series(limpiar_string_coordenadas(x))
)

# Para Actividades (Destino)
df_actividades[['lat_destino', 'lng_destino']] = df_actividades['activity.end.latLng'].apply(
    lambda x: pd.Series(limpiar_string_coordenadas(x))
)

#1. Limpieza en df_visitas
columnas_a_borrar_visitas = ['visit.topCandidate.placeLocation.latLng']
df_visitas = df_visitas.drop(columns=columnas_a_borrar_visitas)

# 2. Limpieza en df_actividades
columnas_a_borrar_actividades = ['activity.start.latLng', 'activity.end.latLng']
df_actividades = df_actividades.drop(columns=columnas_a_borrar_actividades)

In [7]:
df_actividades.head(3)

,startTime,endTime,activity.distanceMeters,activity.topCandidate.type,lat_origen,lng_origen,lat_destino,lng_destino
0,2025-12-28 15:53:04-05:00,2025-12-28 17:05:48-05:00,114.515335,WALKING,4.606785,-74.152691,4.605961,-74.152072
1,2025-12-28 17:38:25-05:00,2025-12-28 20:50:40-05:00,5009.570312,MOTORCYCLING,4.604781,-74.155008,4.625344,-74.121305
2,2025-12-28 21:08:05-05:00,2025-12-29 10:30:56-05:00,4047.625000,IN_PASSENGER_VEHICLE,4.625330,-74.121313,4.606811,-74.152753


In [8]:
df_visitas.head(3)

,startTime,endTime,latitud,longitud
0,2025-12-28 15:53:04-05:00,2025-12-28 17:05:48-05:00,4.609262,-74.152381
1,2025-12-28 17:38:25-05:00,2025-12-28 20:50:40-05:00,4.624256,-74.120272
2,2025-12-28 21:08:05-05:00,2025-12-29 10:30:56-05:00,4.606749,-74.152608


### MAPA DE CALOR UNIFICADO

In [70]:
def extract_new_format_coords(data):
    points = []
    # Buscamos en todos los 'semanticSegments'
    segments = data.get('semanticSegments', [])
    for segment in segments:
        # Las calles recorridas están en 'timelinePath'
        path = segment.get('timelinePath', [])
        for entry in path:
            point_str = entry.get('point', "")
            if point_str:
                try:
                    # Limpiamos caracteres extraños como el símbolo de grado y separamos
                    clean_str = point_str.replace('°', '').replace(' ', '')
                    lat_str, lon_str = clean_str.split(',')
                    points.append([float(lat_str), float(lon_str)])
                except Exception:
                    continue
    return points

print("Procesando tus rutas (Formato 2025)...")

with open(r"C:\Users\9 ----- SIG\Downloads\Rutas.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

coords = extract_new_format_coords(data)

if coords:
    print(f"¡Conseguido! Encontrados {len(coords)} puntos de exploración.")
    df = pd.DataFrame(coords, columns=['lat', 'lon'])
    
    # Crear el mapa centrado en tus coordenadas
    m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
    
    # Ajustamos radius y blur para que se vean bien las calles de tu ciudad
    HeatMap(coords, radius=12, blur=8, min_opacity=0.6, 
            max_zoom=18, max_opacity=0.8, use_local_extrema=True, 
            gradient={0.2: '#0000FF', 0.4: '#00FFFF', 0.6: '#00FF00', 0.8: '#FFFF00', 1: '#FF0000'}).add_to(m)
    
    output_name = 'Mapa_Calles_Exploradas.html'
    m.save(output_name)
    print(f"Archivo '{output_name}' creado con éxito. ¡Ya puedes abrirlo!")
else:
    print("No se pudieron extraer coordenadas. Verifica que el archivo no esté vacío.")

Procesando tus rutas (Formato 2025)...
¡Conseguido! Encontrados 3287 puntos de exploración.
Archivo 'Mapa_Calles_Exploradas.html' creado con éxito. ¡Ya puedes abrirlo!
